In [44]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [76]:
# Get config
import json
with open('./weather-data-retriever-config.json') as config_file:
    config = json.load(config_file)

In [100]:
def process_response(response, location_name, model):
	column_prefix = f"{location_name}-{model}-"
	# Process hourly data. The order of variables needs to be the same as requested.
	hourly = response.Hourly()
	hourly_data = {f"{column_prefix}hourly-{key}": hourly.Variables(i).ValuesAsNumpy() for i, key in enumerate(config["hourly_variables"])}
	hourly_data[f"{column_prefix}hourly-date"] = pd.date_range(
		start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
		end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = hourly.Interval()),
		inclusive = "left"
	)
	# Process daily data. The order of variables needs to be the same as requested.
	daily = response.Daily()
	daily_data = {f"{column_prefix}daily-{key}": daily.Variables(i).ValuesAsNumpy() for i, key in enumerate(config["daily_variables"])}
	daily_data[f"{column_prefix}daily-date"] = pd.date_range(
		start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
		end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = daily.Interval()),
		inclusive = "left"
	)
	daily_data_df = pd.DataFrame(data=daily_data)
	hourly_data_df = pd.DataFrame(data=hourly_data)
	location_df = pd.concat([daily_data_df, hourly_data_df], axis=1)
	return location_df

In [101]:
df = None
for location in config["locations"]:
    for model_i, model in enumerate(config["weather_models"]):
        # Setup the Open-Meteo API client with cache and retry on error
        cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
        retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
        openmeteo = openmeteo_requests.Client(session = retry_session)

        print(f"Getting data for '{location['name']}' and model'{model}'")
        # Make sure all required weather variables are listed here
        # The order of variables in hourly or daily is important to assign them correctly below
        url = "https://api.open-meteo.com/v1/forecast"
        params = {
            "latitude": location["latitude"],
            "longitude": location["longitude"],
            "timezone": location["timezone"],
            "daily": config["daily_variables"],
            "hourly": config["hourly_variables"],
            "models": [model],
        }
        try:
            responses = openmeteo.weather_api(url, params=params)
        except Exception as e:
            print(f"Failed for location '{location["name"]}' and model '{model}' due to: {e}")
            continue
        if len(responses) > 1:
            raise ValueError("More than one response received")
        response = responses[0]
        df_for_model = process_response(response, location_name=location["name"], model=model)
        if model_i == 0:
            df = df_for_model
        else:
            df = pd.concat([df, df_for_model], axis = 1)

Getting data for 'New York' and model'best_match'
Getting data for 'New York' and model'ecmwf_ifs025'
Getting data for 'New York' and model'ecmwf_aifs025'
Getting data for 'New York' and model'cma_grapes_global'
Getting data for 'New York' and model'bom_access_global'
Getting data for 'New York' and model'gfs_seamless'
Getting data for 'New York' and model'gfs_global'
Getting data for 'New York' and model'gfs_hrrr'
Getting data for 'New York' and model'ncep_nbm_conus'
Getting data for 'New York' and model'gfs_graphcast025'
Getting data for 'New York' and model'jma_seamless'
Getting data for 'New York' and model'icon_seamless'
Getting data for 'New York' and model'icon_global'
Getting data for 'New York' and model'icon_eu'
Failed for location 'New York' and model 'icon_eu' due to: {'error': True, 'reason': 'No data is available for this location'}
Getting data for 'New York' and model'icon_d2'
Failed for location 'New York' and model 'icon_d2' due to: {'reason': 'No data is available fo

In [102]:
df


,San Francisco-best_match-daily-temperature_2m_max,San Francisco-best_match-daily-temperature_2m_min,San Francisco-best_match-daily-precipitation_sum,San Francisco-best_match-daily-sunshine_duration,San Francisco-best_match-daily-rain_sum,San Francisco-best_match-daily-showers_sum,San Francisco-best_match-daily-snowfall_sum,San Francisco-best_match-daily-precipitation_hours,San Francisco-best_match-daily-weather_code,San Francisco-best_match-daily-date,...,San Francisco-ukmo_global_deterministic_10km-hourly-rain,San Francisco-ukmo_global_deterministic_10km-hourly-showers,San Francisco-ukmo_global_deterministic_10km-hourly-snowfall,San Francisco-ukmo_global_deterministic_10km-hourly-precipitation_probability,San Francisco-ukmo_global_deterministic_10km-hourly-weather_code,San Francisco-ukmo_global_deterministic_10km-hourly-snow_depth,San Francisco-ukmo_global_deterministic_10km-hourly-visibility,San Francisco-ukmo_global_deterministic_10km-hourly-soil_moisture_0_to_1cm,San Francisco-ukmo_global_deterministic_10km-hourly-sunshine_duration,San Francisco-ukmo_global_deterministic_10km-hourly-date
0,14.606501,9.406500,19.599998,501.713104,19.599998,0.0,0.0,7.0,65.0,2024-12-26 08:00:00+00:00,...,0.0,0.0,0.0,NaN,3.0,NaN,15720.0,NaN,0.0,2024-12-26 08:00:00+00:00
1,14.456500,12.006500,10.500001,25200.000000,10.500001,0.0,0.0,5.0,65.0,2024-12-27 08:00:00+00:00,...,0.0,0.0,0.0,NaN,3.0,NaN,12480.0,NaN,0.0,2024-12-26 09:00:00+00:00
2,16.070000,11.956500,0.000000,11977.780273,0.000000,0.0,0.0,0.0,3.0,2024-12-28 08:00:00+00:00,...,0.4,0.0,0.0,NaN,51.0,NaN,4100.0,NaN,0.0,2024-12-26 10:00:00+00:00
3,15.370001,10.420000,3.500000,19836.939453,3.200000,0.3,0.0,4.0,61.0,2024-12-29 08:00:00+00:00,...,1.3,0.0,0.0,NaN,61.0,NaN,3460.0,NaN,0.0,2024-12-26 11:00:00+00:00
4,12.870001,8.969999,0.000000,30725.480469,0.000000,0.0,0.0,0.0,0.0,2024-12-30 08:00:00+00:00,...,0.8,0.0,0.0,NaN,53.0,NaN,5460.0,NaN,0.0,2024-12-26 12:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2025-01-02 03:00:00+00:00
164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2025-01-02 04:00:00+00:00
165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2025-01-02 05:00:00+00:00
166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2025-01-02 06:00:00+00:00
